# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

Toronto has mainly three different type of clubs, these are The Social Club, the Strip Club and the Jazz Club. This project aims first to create 3 clusters of neighborhoods according to clubs in and around that neighbourhood, secondly recommend a maximum of 3 clubs in each cluster.The target audience of the report is any person/tourist wanting to explore the club life in Toronto.

Toronto is known for its clups, but in choosing the optimum locations; we will aim to detect **locations that atleast have one of the mentioned clubs**, but there should be enough clubs in the vacinity to invite crowds. 

From the data available, candidate neighborhoods will be identified and based on the above criteria  best possible final location will be chosen and suggested. 

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* Clusters that the clubs are grouped into. 
* Proximities of Clubs in different clusters i.e. different type of clubs who are at close proximity to each other will be favoured.
* Number of clubs(Social/Jazz) in Toronto.

Following data sources will be needed to extract/generate the required information:

* Location data specific to Toronto will be scraped of the Wikipedia, this information contains the borough, neighborhood,latitude and longitude.

* All information relating to the type of clubs in the area will be gotten using **Foursquare API**

* Toronto coordinates will be gotten from the internet (a quick google search will be conducted).

In [8]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
from sklearn.cluster import KMeans
import folium
import pandas as pd
import requests
from bs4 import BeautifulSoup as Soup
from pandas.io.json import json_normalize

read=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') #Scrape data from url
soup=Soup(read.text)
soup1=soup.find('tbody')
j=soup1.find_all('tr')

header=[]
upper=[]

for i in soup1.find_all('th'):          #Get header information
    header.append(i.text.strip('\n'))

for i in range(1,len(j)):  #Get data information
    lower=[]
    for td in j[i].find_all("td"):
        lower.append(td.text.strip('\n'))  
    upper.append(lower) 
    
Data=pd.DataFrame(upper,columns=header) #Clean the Dataset
Clean_Data=Data[Data.Borough!='Not assigned'].reset_index(drop=True)
Clean_Data.Neighborhood=Clean_Data.Neighborhood.apply(lambda x: x.replace('/',','))

for i in range(len(Clean_Data.Neighborhood)): #Convert 'Not assigned' Neighborhood to Borough
    if Clean_Data.loc[i,'Neighborhood']=='Not assigned':
        Clean_Data.loc[i,'Neighborhood']=Clean_Data.loc[i,'Borough']
        

df=pd.read_csv('Geospatial_Coordinates.csv')
Canada= Clean_Data.merge(df,how='inner',left_on='Postal code', right_on='Postal Code')
Canada=Canada[['Postal code','Borough','Neighborhood','Latitude','Longitude']]
Canada.rename(columns={'Postal code':'PostalCode'},inplace=True)

In [9]:
Canada

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
